**PRETRAINED MODEL OF YOLOV5 ALONG WITH TRAFFIC LIGHT DETECTION**

In [ ]:
import torch
import cv2
from matplotlib import pyplot as plt

# Load the YOLOv5 pretrained model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')


/usr/local/lib/python3.10/dist-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 121.9 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 5.8s, installed 1 package: ['gitpython>=3.1.30']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-11-13 Python-3.10.12 torch-2.4.1+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 110MB/s] 

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


**Traffic Light Signal Detection: Identifying Dominant Color (Red, Yellow, Green) through Masking and Brightness Analysis**

In [ ]:
# Define the traffic light class ID (COCO class ID for 'traffic light')
traffic_light_class_id = 9

def mask_image_get_brightness_vector(rgb_image):
    resized_image = cv2.resize(rgb_image, (32, 32))
    hsv = cv2.cvtColor(resized_image, cv2.COLOR_RGB2HSV)

    # Extended HSV ranges for red, yellow, and green
    lower_red1, upper_red1 = np.array([0, 60, 40]), np.array([15, 255, 255])
    lower_red2, upper_red2 = np.array([165, 60, 40]), np.array([180, 255, 255])
    lower_yellow, upper_yellow = np.array([15, 80, 80]), np.array([35, 255, 255])
    lower_green, upper_green = np.array([35, 50, 50]), np.array([90, 255, 255])

    mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask_red = cv2.bitwise_or(mask_red1, mask_red2)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    mask_green = cv2.inRange(hsv, lower_green, upper_green)

    masked_red_image = cv2.bitwise_and(resized_image, resized_image, mask=mask_red)
    masked_yellow_image = cv2.bitwise_and(resized_image, resized_image, mask=mask_yellow)
    masked_green_image = cv2.bitwise_and(resized_image, resized_image, mask=mask_green)

    # Calculate brightness vector for each color
    red_brightness_sum = np.sum(masked_red_image[:, :, 0])
    yellow_brightness_sum = np.sum(masked_yellow_image[:, :, 0])
    green_brightness_sum = np.sum(masked_green_image[:, :, 0])

    # Determine the color with the highest brightness
    color_brightness = {'Red': red_brightness_sum, 'Yellow': yellow_brightness_sum, 'Green': green_brightness_sum}
    max_color = max(color_brightness, key=color_brightness.get)
    return max_color

**FOR VIDEO DETECTION**

In [ ]:
import os
import cv2

# Paths for input and output videos
video_path = '/content/drive/MyDrive/output5.mp4'  # Input video path
output_path = '/content/drive/MyDrive/PROJECT_INFOSYS/DETECTED_output1.mp4'  # Output video path
# Set up output directory if it doesn't exist
output_dir = os.path.dirname(output_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


In [ ]:
# Open the input video
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))




In [ ]:
# Define codec and create VideoWriter for the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))


**FOR ONLY TRAFFIC LIGHT DETECTION**

In [ ]:
import numpy as np
# Process all video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert BGR to RGB
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform YOLOv5 inference on the frame
    results = model(img_rgb)

    detected_traffic_lights = []
    color = "Go on"  # Default message if no traffic light is detected

    # Loop over detected objects
    for i, label in enumerate(results.pred[0][:, -1].int()):
        if label == traffic_light_class_id:
            # Get bounding box coordinates
            x1, y1, x2, y2 = map(int, results.pred[0][i, :4])

            # Validate bounding box to ensure it’s within frame bounds
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(frame_width - 1, x2), min(frame_height - 1, y2)

            # Crop the ROI for the traffic light
            roi = img_rgb[y1:y2, x1:x2]

            # Process ROI if it’s non-empty
            if roi.size > 0:
                # Determine the traffic light color
                color = mask_image_get_brightness_vector(roi)

                # Draw bounding box and label color on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, color, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
                detected_traffic_lights.append((x1, y1, x2, y2))

    # Check if no traffic light is detected
    if not detected_traffic_lights:
        message = "NO TRAFFIC SIGNAL"  # Default message when no traffic light is detected

    # Define the message based on traffic light color
    if color == 'Green':
        message = "Go on"
    elif color == 'Red':
        message = "Slow down"
    else:
        message = "Go on"  # Default to "Go on" for other colors or no detection

    # Place the text at the top left of the frame
    cv2.putText(frame, message, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Write the frame to the output video
    out.write(frame)

# Release video capture and writer
cap.release()
out.release()

print("Video processing complete. Output saved to", output_path)

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.am

Video processing complete. Output saved to /content/drive/MyDrive/PROJECT_INFOSYS/DETECTED_output1.mp4


PROCESSED VIDEO LINK:
